`tangle` provides functions for converting markdown to executable source in a semi-lossy manner.  Key design constraints are:

* Lines in the Markdown source must align with the Python source providing useful tracebacks.
* All non-source is compiled and is therefore a string.

In [1]:
import ast, mistune, textwrap, functools, itertools
__all__ = 'markdown_to_python_user_expressions', 'markdown_to_python'

`quote` wrotes non code objects in triple ticks.

In [29]:
    def quote(str, punc=';'):
        str, leading_ws = ''.join(str), []
        lines = str.splitlines(True)
        _ = '"""'
        if _ in str: _ = "'''"
        if not str.strip(): _ = punc = ''
        while lines and (not lines[0].strip()): leading_ws.append(lines.pop(0))    
        str = ''.join(lines)
        end = len(str.rstrip())
        str, ending_ws = str[:end], str[end:]
        if str and str.endswith(_[0]): str += ' '                    
        return F"{''.join(leading_ws)}{_}{str}{_}{punc}{ending_ws}"

`get_first_line` get the first non-`iter`able strings in `lines`

In [30]:
    def get_first_line(lines, line=''):
        for line in lines or ['']: 
            if line.strip(): break
        return line

`get_line_indent` computes the indent of a string.

In [31]:
    def get_line_indent(line):  return len(line) - len(line.lstrip())

The `Markdown` transformer must combine code and non-code.  It must handle logic to properly indent tangled code and non-code.

In [32]:
    class Markdown(mistune.Markdown):
        def render(self, text):
            """Initialize the """
            self.renderer.original = ''.join(text).splitlines()
            self.renderer.final, self.renderer.buffer, self.renderer.min_indent = [], [], 0
            self.renderer.expressions = []
            self.parse(text)
            self.renderer.block_code('', '')
            return '\n'.join(self.renderer.final)

`PythonExpressionsMarkdown` is a `Markdown` transformer that also resolves expressions in inline code. 

> `pidgin` expects the inline code to be expressions, assert is the only statement permitted in in line code because it is idiomatic.

In [33]:
    class PythonExpressionsMarkdown(Markdown):
        def render(self, text): return super().render(text), self.renderer.expressions

The __Lexer__ s only consider coarse features of the markdown spec.  

In [34]:
    class BlockLexer(mistune.BlockLexer): list_rules = default_rules = ['newline', 'block_code', 'list_block', 'paragraph', 'text']
    class InlineLexer(mistune.InlineLexer):  inline_html_rules = default_rules = ['text']

`InlineExpressionsMarkdown` is a special `mistune.Markdown` transformer that records information in __code span__ statatements.

In [35]:
    class InlineExpressionsMarkdown(mistune.Markdown):
        def render(self, text):
            self.renderer.expressions = []
            super().render(text)
            return self.renderer.expressions

`InlineExpressionsRenderer` is a special `mistune.Renderer` transformer that records expressions in `InlineExpressionsRenderer.codespan`s.

In [36]:
    class InlineExpressionsRenderer(mistune.Renderer):
        def codespan(self, text): 
            self.expressions.append(text)
            return super().text(text)

In [37]:
    class Renderer(mistune.Renderer):
        def newline(self):
            self.buffer += ['']
            return ''
        
        def paragraph(self, text):
            self.expressions.extend(InlineExpressionsMarkdown(renderer=InlineExpressionsRenderer()).render(text))
                        
            lines = text.splitlines()
            while lines:
                line = lines.pop()
                while self.original and line not in self.original[0]:
                    self.buffer += [self.original.pop(0)]
                if self.original: self.buffer += [self.original.pop(0)]
                        
            return ''

        def block_code(self, text, lang): 
            if lang:
                return self.paragraph(text)
            
            first_line = get_first_line(text.splitlines())
            
            while self.original and first_line not in self.original[0]:   
                self.buffer += [self.original.pop(0)]
                
            while self.original and not self.original[0].strip(): self.buffer += [self.original.pop(0)]
                
            code, lines = [], text.splitlines()
            while lines and not lines[0].strip(): lines.pop(0)
            
            if self.original:
                while lines:
                    line = lines.pop(0)
                    while line not in self.original[0]:
                        code += [self.original.pop(0)]
                    code += [self.original.pop(0)]
                
            code = '\n'.join(code + [''])
            body = '\n'.join(self.buffer + [''])
            
            last_line = get_first_line(reversed(self.final))
            prior_indent = get_line_indent(last_line)

            definition, returns = last_line.rstrip().endswith(':'), last_line.lstrip().startswith('return')

            this_indent = get_line_indent(get_first_line(code.splitlines()))

            if body.strip() and not self.min_indent: self.min_indent = this_indent
            indent = max(self.min_indent, (returns and min or max)(prior_indent, this_indent))
            indent += 4 * (definition and prior_indent == indent)
            self.final.extend(textwrap.indent(quote(body), ' '*indent).splitlines()) 
            
            self.final.extend(code.splitlines())

            self.buffer = []
            return ''            

## Exports

`markdown_to_python` converts __Markdown__ to __Python__ in a semi lossless way.

In [38]:
def markdown_to_python(str)->"Valid Python Source": 
    renderer = Renderer()
    if str.lstrip().startswith('%%'): return str
    return Markdown(renderer=renderer, 
        inline=InlineLexer(renderer, mistune.InlineGrammar()), 
        block=BlockLexer(mistune.BlockGrammar())).render(str)

`markdown_to_python_user_expressions` is augments `markdown_to_python` and returns any inline code expressions.  

In [39]:
def markdown_to_python_user_expressions(str)->("Valid Python Source", "User Expressions"): 
    renderer = Renderer()
    if str.lstrip().startswith('%%'): return str, []
    return PythonExpressionsMarkdown(renderer=renderer, 
        inline=InlineLexer(renderer, mistune.InlineGrammar()), 
        block=BlockLexer(mistune.BlockGrammar())).render(str)

# Tests

In [25]:
    def test_edge_case():
        return markdown_to_python_user_expressions("""    vega_transforms = "https://vega.github.io/vega-lite/docs/transform.html"
            vega_transforms
            
    """)

In [26]:
    def test_exp():
        source, user_expressions = markdown_to_python_user_expressions('testiong `range` asdfkjadf asdfkjhaskdjfh `print`')
        assert user_expressions

In [27]:
    if 'pytest' in __import__('sys').modules or __name__ == '__main__':
        import hypothesis.strategies
        @hypothesis.given(hypothesis.strategies.text())
        def test_random_strings(str): 
            markdown_to_python_user_expressions(str) 
            
        if __name__ == '__main__': test_random_strings()

In [28]:
    s = """## Docstrings

    A latent feature of `pidgin` is the ability to compose docstrings as markdown.

        class MyClass:
    This is the docstring for `MyClass`.  `pidgin` will automatically wrap this expression in quotes because it follows a class defintion.

        assert MyClass.__doc__


        def my_function(x):
    The same is true for functions.

            return x
            """

    print(markdown_to_python_user_expressions(s)[0])